In [ ]:
from paraview.simple import *
import numpy as np

from vtkGridConstructors import *

# Tested Sun May 16 12:50:07 CEST 2021

In [ ]:
view = GetRenderView()
from ipyparaview.widgets import PVDisplay
pvdisp = PVDisplay(view)
w = display(pvdisp)

# Create a Spherical Grid

In [ ]:
ReqDataScript = """
import numpy as np
from vtk.numpy_interface import algorithms as algs
from vtkGridConstructors import vtkStructuredGridFromArrays

executive = self.GetExecutive()
outInfo = executive.GetOutputInformation(0)
exts = [executive.UPDATE_EXTENT().Get(outInfo, i) for i in range(6)]
whole = [executive.WHOLE_EXTENT().Get(outInfo, i) for i in range(6)]

global_dims = ([whole[1]-whole[0]+1, whole[3]-whole[2]+1, whole[5]-whole[4]+1])

output.SetExtent(exts)

Raxis = np.linspace(1., 2., global_dims[0])[exts[0]:exts[1]+1]

# only use 3/4 of the full longitude in order to view the inside of the sphere
Thetaaxis = np.linspace(0.,np.pi*1.5, global_dims[1])[exts[2]:exts[3]+1]

Phiaxis = np.linspace(0.,np.pi*1.0, global_dims[2])[exts[4]:exts[5]+1]

p, t, r = np.meshgrid(Phiaxis, Thetaaxis, Raxis, indexing="ij")
X = r * np.cos(t) * np.sin(p)
Y = r * np.sin(t) * np.sin(p)
Z = r * np.cos(p)

G = vtkStructuredGridFromArrays()

G.SetCoordinates(X, Y, Z)
G.AddArray(r.ravel(), "radius")
G.AddArray(t.ravel(), "theta")
G.AddArray(p.ravel(), "phi")
# reduce dimension of array by 1 in all directions
G.AddCellArray(p[1:,1:,1:].ravel(), "phi")

V = algs.make_vector(X.ravel(),
                   Y.ravel(),
                   Z.ravel())
                   
# V is a 2D array of shape(np.prod(global_dims), 3)
G.AddArray(V, "Velocity")

# reduce dimension of array by 1 in all directions, except the last to keep velocity a 3-tuple
Vcell = V.reshape(global_dims[2],global_dims[1],global_dims[0],3)[1:,1:,1:,:].reshape((global_dims[0]-1)*(global_dims[1]-1)*(global_dims[2]-1), 3)
G.AddCellArray(Vcell, "Velocity")

output.ShallowCopy(G.GetOutput())
"""

ReqInfoScript = """
executive = self.GetExecutive()
outInfo = executive.GetOutputInformation(0)
# A 3D spherical mesh
dims = [5, 9, 7] # radius, theta(east-west longitude), phi(north-south latitude)
outInfo.Set(executive.WHOLE_EXTENT(), 0, dims[0]-1 , 0, dims[1]-1 , 0, dims[2]-1)
outInfo.Set(vtk.vtkAlgorithm.CAN_PRODUCE_SUB_EXTENT(), 1)
"""

SphericalGrid = ProgrammableSource(registrationName="SphericalGrid")
SphericalGrid.OutputDataSetType = 'vtkStructuredGrid'
SphericalGrid.Script = ReqDataScript
SphericalGrid.ScriptRequestInformation = ReqInfoScript

In [ ]:
rep0 = Show(SphericalGrid, view)
rep0.Representation = 'Surface With Edges'
ColorBy(rep0,"radius")

In [ ]:
ResetCamera()

In [ ]:
Hide(SphericalGrid)

# A cartesian grid, a.k.a. vtkImageData

In [ ]:
dims = [64,64,64]
bounds = (-10.0, 10.0, -10.0, 10.0,-10.0, 10.0)
xaxis = np.linspace(bounds[0], bounds[1], dims[0])
yaxis = np.linspace(bounds[2], bounds[3], dims[1])
zaxis = np.linspace(bounds[4], bounds[5], dims[2])
[xc,yc,zc] = np.meshgrid(zaxis,yaxis,xaxis, indexing="ij")
data = xc**2 + yc**2 + zc**2

from vtk.util.vtkImageImportFromArray import vtkImageImportFromArray
iD = vtkImageImportFromArray()
iD.SetArray(data) # it is given the name "scalars" by default
iD.SetDataOrigin((bounds[0], bounds[2], bounds[4]))
iD.SetDataSpacing(((bounds[1]-bounds[0])/(dims[0]-1),
                   (bounds[3]-bounds[2])/(dims[0]-1),
                   (bounds[5]-bounds[4])/(dims[0]-1)))
iD.Update()
iD.GetOutput().GetPointData().SetActiveScalars("scalars")

print("mesh dimensions = ", iD.GetOutput().GetDimensions())
assert iD.GetOutput().GetPointData().GetArray(0).GetName() == 'scalars'
datarange = iD.GetOutput().GetPointData().GetArray(0).GetRange()
print("data scalar range = ", datarange)

# create a new 'PVTrivialProducer'
CartesianGrid = PVTrivialProducer(registrationName="CartesianGrid")
obj = CartesianGrid.GetClientSideObject()
obj.SetOutput(iD.GetOutput())
CartesianGrid.UpdatePipeline()

rep0 = Show(CartesianGrid)
rep0.Representation = 'Surface With Edges'

In [ ]:
ColorBy(rep0, "scalars")
rep0.Representation = 'Surface'

In [ ]:
ResetCamera()

In [ ]:
Hide(CartesianGrid)

# Create a cylindrical - structured grid

In [ ]:
dims = [13,27,15] # Radius dimension, Theta dimension, Z dimension
Raxis = np.linspace(1., 2., dims[0])
Thetaaxis = np.linspace(0.,np.pi*1.5, dims[1])
Zaxis = np.linspace(0., 2.0, dims[2])
Z, t, r = np.meshgrid(Zaxis, Thetaaxis, Raxis, indexing="ij")
X = r * np.cos(t)
Y = r * np.sin(t)

NsG = vtkStructuredGridFromArrays()
NsG.SetCoordinates(X, Y, Z)
NsG.AddArray(r.ravel(), "radius")
NsG.AddArray(Z.ravel(), "z-elevation")
NsG.AddArray(t.ravel(), "angle")

print("StructuredGrid mesh dimensions = ", NsG.GetDimensions())
print("StructuredGrid mesh bounds = ", NsG.GetOutput().GetBounds())
assert NsG.GetOutput().GetPointData().GetArray(1).GetName() == 'z-elevation'
datarange = NsG.GetOutput().GetPointData().GetArray("z-elevation").GetRange()
print("StructuredGrid data scalar range = ", datarange)

CylindricalGrid = PVTrivialProducer(registrationName="CylindricalGrid")
obj = CylindricalGrid.GetClientSideObject()
obj.SetOutput(NsG.GetOutput())
CylindricalGrid.UpdatePipeline()

rep2 = Show(CylindricalGrid)
rep2.Representation = 'Surface With Edges'

ColorBy(rep2, "radius")

In [ ]:
ResetCamera()

In [ ]:
Hide(CylindricalGrid)

# Create a point cloud

In [ ]:
nbpts = 10000
X = np.random.random_sample((nbpts, 3))

PC = vtkPointSetFromArrays()
#PC.SetCoordinates(X[:,0], X[:,1],X[:,2])
PC.SetVCoordinates(X)
PC.AddArray(X[:,0]*X[:,0] + X[:,1]*X[:,1] + X[:,2]*X[:,2], "distancesq")
PC.AddArray(X[:,0], "X")

print("Point Cloud has", PC.GetOutput().GetNumberOfPoints(), "vertices")
# create a new 'PVTrivialProducer'
PointCloud = PVTrivialProducer(guiName="Point Cloud")
obj = PointCloud.GetClientSideObject()
obj.SetOutput(PC.GetOutput())
PointCloud.UpdatePipeline()

rep3=Show(PointCloud)
rep3.Representation = 'Points'
ColorBy(rep3, 'distancesq')

In [ ]:
ResetCamera()

In [ ]:
Hide(PointCloud)

# Create a Rectilinear Grid

In [ ]:
dims = [64,64,64] # Radius dimension, Theta dimension, Z dimension
xaxis = np.linspace(0., 1., dims[0])
xaxis = xaxis**2
yaxis = np.linspace(0., 1., dims[1])
yaxis = np.sqrt(yaxis)
zaxis = np.linspace(0., 1., dims[2])

[xc,yc,zc] = np.meshgrid(zaxis,yaxis,xaxis, indexing="ij")
data = np.sqrt(xc**2 + yc**2 + zc**2)

rG = vtkRectGridFromArrays()
rG.SetCoordinates(xaxis, yaxis, zaxis)
rG.AddArray(data.ravel(), "scalars")

print("RectilinearGrid dimensions = ", rG.GetDimensions())
assert rG.GetOutput().GetPointData().GetArray(0).GetName() == 'scalars'
datarange = rG.GetOutput().GetPointData().GetArray("scalars").GetRange()
print("RectilinearGrid  data scalar range = ", datarange)

# create a new 'PVTrivialProducer'
RectilinearGrid = PVTrivialProducer(registrationName="RectilinearGrid")
obj = RectilinearGrid.GetClientSideObject()
obj.SetOutput(rG.GetOutput())
RectilinearGrid.UpdatePipeline()

rep1 = Show(RectilinearGrid)
rep1.Representation = 'Surface With Edges'
ColorBy(rep1, 'scalars')

In [ ]:
ResetCamera()

In [ ]:
rep1.Representation = 'Surface'